In [70]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [71]:
webpage=requests.get('https://crex.live/fixtures/match-list').text

In [72]:
soup=BeautifulSoup(webpage,'lxml')

In [73]:
matches=soup.find_all(class_='matches-card-space')
dates=soup.find_all(class_='date')

In [74]:
len(matches)

2

In [75]:
dates

[<div _ngcontent-sc125="" class="date"><div _ngcontent-sc125="">Tue, 12 Nov 2024</div><!-- --><!-- --></div>,
 <div _ngcontent-sc125="" class="date"><div _ngcontent-sc125="">Wed, 13 Nov 2024</div><!-- --><!-- --></div>]

In [76]:
def scrape_scorecard(url):
    scorecard_page = requests.get(url).text
    scorecard_soup = BeautifulSoup(scorecard_page, 'lxml')

    scorecard = scorecard_soup.find_all('table')
    if scorecard:
        print('Scorecard Details:')
        for score in scorecard:
            print(f" {score.text.split('/n')}")
    else:
        print("No scorecard available")

In [ ]:
for i in range(len(dates)):
    print()
    print(dates[i].text.strip())
    print()

    if i < len(matches):
        match_details = matches[i].find_all('div', class_='match-details')

        for match in match_details:
            tn = []
            ts = []
            to = []

            team_name_elements = match.find_all('span', class_='team-name')
            if len(team_name_elements) >= 2:
                for k in range(2):
                    tn.append(team_name_elements[k].text)

                    if match.find(class_='not-started'):
                        continue
                    else:
                        team_score_elements = match.find_all('span', class_='team-score')
                        if len(team_score_elements) > k:
                            ts.append(team_score_elements[k].text)
                        else:
                            ts.append("N/A")

                        overs_list = match.find_all('span', class_='total-overs')
                        if len(overs_list) > k:
                            to.append(overs_list[k].text)
                        else:
                            to.append("-")

                print(f'Teams:\t{tn[0]}\tVs\t{tn[1]}')

                if match.find(class_='not-started'):
                    print("Status: Match not Yet Started")
                else:
                    print(f'Score:\t{ts[0]}\t\t{ts[1]}')
                    if 'Yet to bat' in ts:
                        if ts[0] == 'Yet to bat':
                            print(f'Overs:\t-\t\t{to[0]}')
                        else:
                            print(f'Overs:\t{to[0]}\t\t-')
                    else:
                        print(f'Overs:\t{to[0]}\t\t{to[1]}')

                live_tag = match.find('div', class_='live-info')
                result_tag = match.find('div', class_='result')

                if live_tag:
                    result = live_tag.find('span').text.strip()
                    print(f"Status: {result}")
                elif result_tag:
                    result = result_tag.find('span').text.strip()
                    print(f"Result:\t{result}")
                    tour = match.find('span', class_='reason').text.strip() if match.find('span', 'reason') else "-"
                    print(f"Tournament Name: {tour}")
                else:
                    time = match.find('div', class_='start-text').text.strip() if match.find('div', 'start-text') else "-"
                    print(f"Timing: {time}")
                    tour = match.find('p', class_="time").text.strip() if match.find('p', 'time') else "-"
                    print(f"Tournament Name: {tour}")

                match_link_tag = match.find_parent('a', class_='match-card-wrapper')
                if match_link_tag:
                    match_link = match_link_tag['href']
                    full_link = f"https://crex.live{match_link}"
                    print(f"Scorecard Link: {full_link}")
                    scrape_scorecard(full_link)
                print()



Tue, 12 Nov 2024

Teams:	MSW	Vs	MAR
Score:	161/3		102/6
Overs:	10.0		10.0
Result:	MSW Won
Tournament Name: 86th T10, ECS Malta 2024
Scorecard Link: https://crex.live/scoreboard/RQL/1OX/86th-Match/OX/OZ/mar-vs-msw-86th-match-european-cricket-series-malta-2024/scorecard
Scorecard Details:
 ['BatterRB4s6sSRMuhammad Suleman c Soman b Johny 8110172.73Umair Saleem c Dianish b Anoop 20703285.71Nowell Khosla(C) c Dianish b Soman 341333261.54Fanyan Mughal lbw b Johny  261122236.36Adnan Hassan NOT OUT 260033.33Arslan Arshad c Johny b Sukumaran 150020.00Sobhan Sarkar c Abraham b Thomas 350060.00Waqas Khan NOT OUT 2200100.00']
 ['BowlerOMRWERAjin Soman2.001015.00Abishek Anoop2.0047123.50Tom Thomas2.0022111.00Sajith Sukumaran2.001618.00Jithin Johny2.00623.00']
 ['BatsmanScoreOversUmair Saleem25-11.5Muhammad Suleman61-24.1Nowell Khosla(C)91-36.2Fanyan Mughal95-47.3Arslan Arshad96-58.4Sobhan Sarkar99-69.3']

Teams:	MMA	Vs	RDS
Score:	72/1		70/7
Overs:	4.4		10.0
Result:	MMA Won
Tournament Name: 87th T